In [1]:
import pandas.io.data as web
import pandas as pd
import datetime
import Markowitz
import numpy as np
import math

In [2]:
class Position:
    def __init__(self,symbol,direction,in_price,start_date,weight,exchange,asset_class):
        #format for dates: "YYYY-MM-dd"
        self.symbol = symbol
        self.direction = direction
        self.in_price = in_price
        self.start_date = start_date
        self.exchange = exchange
        self.asset_class = asset_class
        self.weight = weight
        request = web.DataReader(symbol,'yahoo',datetime.datetime(2010, 1, 1))['Close']
        self.returns = request.pct_change()[1:len(request)-1]
        if direction == "long":
            self.returns *= 100
        else:
            self.returns *= -100

In [42]:
class Portfolio:
    def __init__(self,assets):
        self.positions = assets
        self.symbols = [asset.symbol for asset in self.positions]
        self.returns_grid = np.array([asset.returns for asset in self.positions])
        self.weights = np.array([asset.weight for asset in self.positions])
        self.returns = np.dot(self.weights.transpose(),self.returns_grid)
        self.covariance_matrix = np.cov(self.returns_grid)
        self.net_variance = np.dot(self.weights.T,np.dot(self.covariance_matrix,self.weights))
        self.net_expectation = np.mean(self.returns)
        
    def optimal_portfolio(self):
        #looking back one year only. subject to change
        start_index = len(self.returns)-253
        end_index = len(self.returns)-1
        return_grid = self.returns_grid[:,start_index:end_index]
        weights, poly = Markowitz.optimal_portfolio_quad(return_grid,frontier=True)
        return {'weights':weights,'curve':poly}

### Example : use of the data structure and efficiency frontier

In [37]:
import matplotlib.pyplot as plt
%matplotlib inline
tickers = ['YHOO','STK','GOOGL','AMZN','EBAY']
weights = [0.2,0.1,0.4,0.15,0.15]
#instantiate positions
positions = []
start_date = datetime.datetime(2010, 1, 1)
for i in range(len(tickers)):
    positions.append(Position(tickers[i],"long",30,start_date,weights[i],"NASDAQ","stock"))
#instantiate portfolio
portfolio = Portfolio(positions)

struct = portfolio.optimal_portfolio()
x1 = struct['curve'][2]
x2 = struct['curve'][1]
x3 = struct['curve'][0]
returns = np.linspace(-5,5,1000)
risks = [x1+x2*returns[i]+x3*returns[i]*returns[i] for i in range(len(returns))]

plt.plot(risks,returns)